In [377]:
using JuMP, MosekTools, Gurobi
using SparseArrays, LinearAlgebra, Plots
using PowerModels, Ipopt
include("../src/CVXRS_OPF.jl")

Academic license - for non-commercial use only


LoadError: syntax: missing comma or ) in argument list

In [379]:
num_data=100#00

network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case3_lmbd.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case5_pjm.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case14_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case24_ieee_rts.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case30_as.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case30_fsr.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case30_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case39_epri.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case57_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case73_ieee_rts.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case89_pegase.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case118_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case162_ieee_dtc.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case179_goc.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case200_tamu.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case240_pserc.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case300_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case500_tamu.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case588_sdet.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1354_pegase.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1888_rte.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1951_rte.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case2000_tamu.m");

result_opf=run_opf(network_data, ACPPowerModel, with_optimizer(Ipopt.Optimizer,print_level=0))
println("Local search OPF objective solution: ",round(result_opf["objective"],digits=2))

## Initialize OPF
network_data=opf_initialization(network_data, 0.0)
#[network_data["gen"][i]["ptc_factor"]=Int(network_data["bus"][string(gen["gen_bus"])]["bus_type"]==3) for (i,gen) in network_data["gen"]]
[network_data["gen"][i]["ptc_factor"]=gen["gen_status"]*float((gen["pmax"]-gen["pmin"])>0) for (i,gen) in network_data["gen"]]

network_data=runpf(network_data)
test_runpf(network_data)
println("Initial OPF objective solution: ",round(network_data["cost"],digits=2))
violation_status, margin=check_violation(network_data)




[info | PowerModels]: removing 3 cost terms from generator 3: Float64[]
Local search OPF objective solution: 5812.64
Initial OPF objective solution: 5812.64


(Dict("line"=>[],"qg"=>[],"pg"=>[],"angle"=>[],"vmag"=>[]), 0.1)

In [356]:

(Σ0,γ0)=network_data["uncertainty"]
Σ0=Σ0+1e-3*Matrix{Float64}(I,size(Σ0,1),size(Σ0,1))
network_data["uncertainty"]=(Σ0,gamma_req)
pl_sampled,data_gencost_bounded,violation_count_bounded,sample_violation_bounded=sample_pf(network_data,num_data,1)
count_bounded=violation_count_bounded
cost_worst_sampled=maximum(data_gencost_bounded)

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req/2)
pl_sampled,data_gencost_bounded,violation_count_normal,sample_violation_bounded=sample_pf(network_data,num_data,0)
count_normal=violation_count_normal

println("Violation probability (Bounded): ",count_bounded/num_data*100)
println("Violation probability (Normal): ",count_normal/num_data*100)


Violation probability (Bounded): 100.0
Violation probability (Normal): 100.0


In [382]:
include("../src/CVXRS_OPF.jl")

gamma_req=0.01
max_iter_SCRS=1

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req)

network_data, result_cvxr=scrs(network_data, max_iter_SCRS,false);

println("Robust OPF objective solution: ",round(network_data["cost"],digits=2))





Academic license - for non-commercial use only
Iteration 

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{MosekModel}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/dc/.julia/packages/MathOptInterface/A2UPd/src/Utilities/copy.jl:140


1: 5829.95  5892.815   OPTIMAL    0.0017   0.0
Robust OPF objective solution: 5829.95


In [376]:

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req)
pl_sampled,data_gencost_bounded,violation_count_bounded,sample_violation_bounded=sample_pf(network_data,num_data,1)
count_bounded=violation_count_bounded
cost_worst_sampled=maximum(data_gencost_bounded)

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req/2)
pl_sampled,data_gencost_bounded,violation_count_normal,sample_violation_bounded=sample_pf(network_data,num_data,0)
count_normal=violation_count_normal

println("Violation probability (Bounded): ", count_bounded/num_data*100)
println("Violation probability (Normal): ", count_normal/num_data*100)
println("Solver Time: ", sum(result_cvxr["solver_time"]))
println("Solver Time (1 iter): ", result_cvxr["solver_time"][2])


Violation probability (Bounded): 70.0
Violation probability (Normal): 77.0
Solver Time: 23.386204957962036
Solver Time (1 iter): 23.386204957962036
